In [ ]:
import numpy as np
import pandas as pd
import hvplot.pandas
from datashader.utils import lnglat_to_meters
import holoviews as hv
from holoviews.element import tiles 
import matplotlib
import plotly.express as px
import panel as pn
from IPython.display import HTML, display, clear_output
from astropy import units as u
from astropy.coordinates import SkyCoord
from holoviews.plotting.util import process_cmap
from colorcet import fire


In [ ]:
pn.extension()

In [ ]:
stars = pd.read_csv("data/stars.csv")
# sample from only brightest stars for background
stars = stars[stars["phot_g_mean_mag"]>11].sample(frac=0.1)

In [ ]:
exoplanets = pd.read_csv("data/exoplanets.csv")
candidates = pd.read_csv("data/candidates.csv").sample(frac=0.05)

# define conversion functions w/ astropy

def eqtogalB(a,b):
    ret = SkyCoord(ra=a*u.degree,dec=b*u.degree,frame='icrs').galactic
    return float(ret.to_string("decimal").split( )[0])

def eqtogalL(a,b):
    ret = SkyCoord(ra=a*u.degree,dec=b*u.degree,frame='icrs').galactic
    return float(ret.to_string("decimal").split( )[1])

# add new columns with converted coordinates

stars['b'] = pd.Series(eqtogalB(a,b) for (a,b) in zip(stars["ra"],stars["dec"]))
stars['l'] = pd.Series(eqtogalL(a,b) for (a,b) in zip(stars["ra"],stars["dec"]))

# note: this takes forever for stars

exoplanets['b'] = pd.Series(eqtogalB(a,b) for (a,b) in zip(exoplanets["ra"],exoplanets["dec"]))
exoplanets['l'] = pd.Series(eqtogalL(a,b) for (a,b) in zip(exoplanets["ra"],exoplanets["dec"]))


candidates['b'] = pd.Series(eqtogalB(a,b) for (a,b) in zip(candidates["ra"],candidates["dec"]))
candidates['l'] = pd.Series(eqtogalL(a,b) for (a,b) in zip(candidates["ra"],candidates["dec"]))



In [ ]:
def select_year(year=0):
    row = exoplanets.loc[dfp["disc_year"]==year]
    return row.style.format({"time": lambda t: t.strftime("%c")})

In [ ]:
year_slider = pn.widgets.RangeSlider(name='Discovery year range', start=1996, end=2021)


In [ ]:
checkbox_unconfirmed = pn.widgets.Checkbox(name='Show sampling of uncomfirmed exoplanets')

checkbox_habitable = pn.widgets.Checkbox(name='Show only planets in potentially habitable zone')


In [ ]:
select_size = pn.widgets.Select(name='Size points by:', options={"Earth radius":"radius", "Earth mass":"mass"})

select_color = pn.widgets.Select(name='Color points by:', options={"Earth radius":"radius", "Earth mass":"mass", "Temperature": "temperature"})


In [ ]:
# put a dot for sun
d = {'b':[0],'l':[0]}
origin = pd.DataFrame(data=d)


In [ ]:
# stars aren't really obvious for background -- why?
@pn.depends(year_slider, checkbox_unconfirmed, checkbox_habitable, select_size, select_color)

def filter_df(year_range, checkbox_unconfirmed, checkbox_habitable, select_size, select_color):
    exo_lower = exoplanets.disc_year>=year_range[0]
    exo_upper = exoplanets.disc_year<=year_range[1]
    hab = exoplanets.habitable == True
    
    if checkbox_habitable:
        exo_filter = exo_lower & exo_upper & hab
    else:
        exo_filter = exo_lower & exo_upper
        
    can_lower = candidates.year>=year_range[0]
    can_upper = candidates.year<=year_range[1]
    can_filter = can_lower & can_upper

    if checkbox_unconfirmed:
        star_background = (stars.hvplot.scatter(x='b',y='l',datashade=True,
                                                       color="phot_g_mean_mag",cmap=fire,
                                                       colorbar=True))
        overlay_points = (exoplanets[exo_filter].hvplot.scatter(x='b',y='l',color=select_color,
                                                       xlabel='longitude (deg)',
                                                       ylabel='latitude (deg)',
                                                       clabel=select_color).opts(cmap='blues',
                                                       size=hv.dim(select_size)))
        
        # there's probably a smarter way to make the circles smaller for mass
        if select_size == "mass":
            overlay_points.opts(size = 0.01*hv.dim(select_size))
        candidate_points = (candidates[can_filter].hvplot.scatter(x='b',y='l',
                                                       size=30,color="#33FF36",alpha=0.5).opts(cmap='greens',
                                                       cnorm='log',bgcolor="black"))
        return star_background*origin.hvplot.scatter(x='b',y='l',size=60,color="yellow")*overlay_points*candidate_points
    else:
        overlay_points = (exoplanets[exo_filter].hvplot.scatter(x='b',y='l', color=select_color,
                                                       xlabel='longitude (deg)',ylabel='latitude (deg)',
                                                       clabel=select_color).opts(cmap='blues',
                                                        size=hv.dim(select_size)))
        star_background = (stars.hvplot.scatter(x='b',y='l',
                                                       datashade=True,color="phot_g_mean_mag",cmap=fire,
                                                        colorbar=True)).opts(bgcolor="black")
        if select_size == "mass":
            overlay_points.opts(size = 0.01*hv.dim(select_size))
            
        return star_background*origin.hvplot.scatter(x='b',y='l',size=60,color="yellow")*overlay_points
    
    
        
# make habitable points more distinguishable and obvious
def radius_mass():
    habitable_points = exoplanets[exoplanets['habitable']==True].hvplot.scatter(x='mass',y='radius',color="red",label="Potentially habitable",size=30)
    uninhabitable_points = exoplanets[exoplanets['habitable']==False].hvplot.scatter(x='mass',y='radius',color="blue",alpha=0.5,label="Uninhabitable",size=10)
    return uninhabitable_points*habitable_points
# what to do about all the data points without both mass and radius
filtered_view = pn.Row(
    pn.Column(year_slider, select_size, select_color, checkbox_unconfirmed, checkbox_habitable, pn.panel(filter_df, width=800),pn.Row(pn.panel(radius_mass,width=400)))
)

filtered_view

